<a href="https://colab.research.google.com/github/axel-sirota/normalise-data-pandas/blob/main/module4/NormaliseDataPandas_Mod4Demo2_DropDuplicates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dropping duplicates



## Prep

In the last series of demos we worked with `Nans`, now we are going to work with another troublesome issue of datasets: duplicated data. For this we will use a slightly midified version of the drinks dataset tha randomly duplicated rows.

In [1]:
%%writefile get_data.sh
if [ ! -f drinks_duplicated.csv ]; then
  wget -O drinks_duplicated.csv https://raw.githubusercontent.com/axel-sirota/normalise-data-pandas/main/data/drinks_duplicated.csv
fi

Overwriting get_data.sh


In [2]:
!bash get_data.sh

In [3]:
import numpy as np
import pandas as pd

drinks_duplicated =  pd.read_csv('drinks_duplicated.csv')
drinks_duplicated

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
0,Malta,149,100,120,6.6,EU
1,Slovakia,196,293,116,11.4,EU
2,Brunei,31,2,1,0.6,AS
3,Cameroon,147,1,4,5.8,AF
4,Bahamas,122,176,51,6.3,NaN
...,...,...,...,...,...,...
324,Denmark,224,81,278,10.4,EU
325,Mexico,238,68,5,5.5,NaN
326,Bolivia,167,41,8,3.8,SA
327,Brazil,245,145,16,7.2,SA


## Droppin duplicates

The easiest way to drop duplicates is the one we have seen:

- Create the indexer with duplicated()
- Filter by that:

In [4]:
drinks_duplicated.duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
324    False
325    False
326    False
327     True
328    False
Length: 329, dtype: bool

In [5]:
drinks_duplicated[~drinks_duplicated.duplicated()]

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
0,Malta,149,100,120,6.6,EU
1,Slovakia,196,293,116,11.4,EU
2,Brunei,31,2,1,0.6,AS
3,Cameroon,147,1,4,5.8,AF
4,Bahamas,122,176,51,6.3,NaN
...,...,...,...,...,...,...
319,Morocco,12,6,10,0.5,AF
324,Denmark,224,81,278,10.4,EU
325,Mexico,238,68,5,5.5,NaN
326,Bolivia,167,41,8,3.8,SA


The only issue, is that this can easily not drop rows you don't care about, but they are not exactly equal. Let's create such a scenario: 

In [13]:
amount_to_duplicate=20
cache = []
new_df_list = []
new_row = []
for row in drinks_duplicated[~drinks_duplicated.duplicated()].iterrows():
  if row[0]< 10:
    continue
  element = list(row[1])
  if amount_to_duplicate > 0:
    cache.append(element)
    if len(cache) > 5:
      element[2:4] = cache.pop()[2:4]
    amount_to_duplicate -= 1
  new_df_list.append(element)
df = pd.DataFrame(new_df_list, columns = drinks_duplicated.columns)

Here we created a DataFrame such that quite randomly with a cache we duplicate the 2nd to fourth columns, therefore we shouldn't have any single duplicate

In [14]:
df

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
0,Seychelles,157,25,51,4.1,AF
1,Russian Federation,247,326,73,11.5,AS
2,Lithuania,343,244,56,12.9,EU
3,Macedonia,106,27,86,3.9,EU
4,Equatorial Guinea,92,0,233,5.8,AF
...,...,...,...,...,...,...
179,Morocco,12,6,10,0.5,AF
180,Denmark,224,81,278,10.4,EU
181,Mexico,238,68,5,5.5,NaN
182,Bolivia,167,41,8,3.8,SA


In [16]:
sum(df.duplicated())

0

Indeed it is the case. Now let's filter out the rows such that spirit and wine servings are equal, we can do it the following way:

In [17]:
df.drop_duplicates(subset=['spirit_servings', 'wine_servings'])

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
0,Seychelles,157,25,51,4.1,AF
1,Russian Federation,247,326,73,11.5,AS
2,Lithuania,343,244,56,12.9,EU
3,Macedonia,106,27,86,3.9,EU
4,Equatorial Guinea,92,0,233,5.8,AF
...,...,...,...,...,...,...
178,Syria,5,35,16,1.0,AS
179,Morocco,12,6,10,0.5,AF
180,Denmark,224,81,278,10.4,EU
181,Mexico,238,68,5,5.5,NaN


Notice the amount of rows reduced! We can also drop both rows on a subset of columns

In [18]:
df.drop_duplicates(subset=['spirit_servings', 'wine_servings'], keep=False)

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
0,Seychelles,157,25,51,4.1,AF
1,Russian Federation,247,326,73,11.5,AS
2,Lithuania,343,244,56,12.9,EU
3,Macedonia,106,27,86,3.9,EU
4,Equatorial Guinea,92,0,233,5.8,AF
...,...,...,...,...,...,...
178,Syria,5,35,16,1.0,AS
179,Morocco,12,6,10,0.5,AF
180,Denmark,224,81,278,10.4,EU
181,Mexico,238,68,5,5.5,NaN
